### Importing the necessary libraries and packages

In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Loading the Dataset

In [3]:
df = pd.read_csv('https://grantmlong.com/data/titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


* Dropping usessential features `PassengerId`, `Name`, `Ticket`, and `Cabin`.
* Imputing NaN and Null data.

In [5]:
df.drop(['Cabin', 'PassengerId', 'Name', 'Ticket'], axis = 1, inplace = True)


imp = SimpleImputer(missing_values=np.nan, strategy='mean', fill_value=None)
df['Age'] = imp.fit_transform(df[['Age']]).ravel()

catimp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df['Embarked'] = catimp.fit_transform(df[['Embarked']]).ravel()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


### Classifying Age and Fare into different buckets/classes.

In [7]:
df['Age'] = df['Age'].apply(lambda x: "Child" if x <= 13 else ("Teenager" if x < 18 else ("Adult" if x < 60 else "Senior Citizen")))

In [8]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,Adult,1,0,7.2500,S
1,1,1,female,Adult,1,0,71.2833,C
2,1,3,female,Adult,0,0,7.9250,S
3,1,1,female,Adult,1,0,53.1000,S
4,0,3,male,Adult,0,0,8.0500,S


In [9]:
df["Fare"] = df['Fare'].apply(lambda x : "Very Cheap" if x // 25 == 0.0 else ("Cheap" if x // 25 == 1.0 else ("Mid-range" if x // 25 <= 3 else ("Expensive" if x // 25 <= 6 else "Very Expensive"))))

In [10]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,Adult,1,0,Very Cheap,S
1,1,1,female,Adult,1,0,Mid-range,C
2,1,3,female,Adult,0,0,Very Cheap,S
3,1,1,female,Adult,1,0,Mid-range,S
4,0,3,male,Adult,0,0,Very Cheap,S


### Separating Positive Examples

In [11]:
df_positive = df.loc[df['Survived'] == 1].drop('Survived', axis = 1)
df_positive.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
1,1,female,Adult,1,0,Mid-range,C
2,3,female,Adult,0,0,Very Cheap,S
3,1,female,Adult,1,0,Mid-range,S
8,3,female,Adult,0,2,Very Cheap,S
9,2,female,Teenager,1,0,Cheap,C


### Implementing Find - S algorithm

In [12]:
def findS(data):
  # Initialize hypothesis to the first row/input tuple of the dataset.
  h = data.iloc[0, :]

  # Print data
  print(data)

  # Iterate through all examples/tuples to find a more general hypothesis
  cols = data.columns
  for index, row in data.iterrows():
    for i in range(len(cols)):
      if h[i] != '?':
        if h[i] != row[cols[i]]:
          h[i] = '?'

  return tuple(h)

### Getting the hypothesis for a few examples

In [13]:
findS(data = df_positive[:5])

   Pclass     Sex       Age  SibSp  Parch        Fare Embarked
1       1  female     Adult      1      0   Mid-range        C
2       3  female     Adult      0      0  Very Cheap        S
3       1  female     Adult      1      0   Mid-range        S
8       3  female     Adult      0      2  Very Cheap        S
9       2  female  Teenager      1      0       Cheap        C


('?', 'female', '?', '?', '?', '?', '?')

In [14]:
findS(data = df_positive[15: 20])

    Pclass     Sex       Age  SibSp  Parch        Fare Embarked
31       1  female     Adult      1      0   Expensive        C
32       3  female     Adult      0      0  Very Cheap        Q
36       3    male     Adult      0      0  Very Cheap        C
39       3  female  Teenager      1      0  Very Cheap        C
43       2  female     Child      1      2       Cheap        C


('?', '?', '?', '?', '?', '?', '?')

In [15]:
findS(data = df_positive[30:35])

    Pclass     Sex       Age  SibSp  Parch        Fare Embarked
68       3  female  Teenager      4      2  Very Cheap        S
74       3    male     Adult      0      0   Mid-range        S
78       2    male     Child      0      2       Cheap        S
79       3  female     Adult      0      0  Very Cheap        S
81       3    male     Adult      0      0  Very Cheap        S


('?', '?', '?', '?', '?', '?', 'S')